# Klassische Segmentierung
Unter klassischer Segmentierung verstehen wir Methoden der Bildanalyse, die nicht auf den Konzepten von Deep Learning basieren. Hier werden meist Filter und Thresholds verwendet. Hinzu kommen mathematische Operationen mit Bildern und morphologische Operationen.
### Filter
Bei einem Filter wird das Bild in irgendeiner weise weiterverarbeitet. Filter sind meist lokal, d.h. der Wert des neuen Bildes hängt nicht von allen Pixeln des Ausgangsbildes ab, sondern nur von wenigen benachbarten Pixeln. Meist werden Filter daher über Faltungen definiert. Dies sind lineare Filter, bspw. Mean- oder Gauß-Filter. Es gibt jedoch auch Min- oder Median-Filter. Diese sind auch über einen Kernel definiert, können jedoch nicht als Faltung dargestellt werden. Darüberhinaus sind diese nicht-linear.
### Thresholds
Mit Hilfe von Thresholds können Bilder diskretisiert werden. Meist werden sie binärisiert. D.h. aus einem Graustufenbild entsteht ein Bild, in dem jedes Pixel nur noch an oder aus sein kann. Für das Thresholding wird meist ein Wert vorgegeben, alle Pixel, deren Intensität darunter liegt, werden auf 0, alle anderen auf 1 gesetzt. Es gibt verschiedene Methoden, um diesen Wert automatisiert zu bestimmen. Meist basieren diese Methoden auf dem Histogram eines Bildes.
### Mathematische Operationen
Oftmals werden Bilder gefiltert und dann mit einander addiert oder multipliziert. So kann bspw. Rauschen oder das Hintergrundsignal eliminiert werden. Bei diesen Operationen sollte unbedingt auf den Datentyp der Bilder geachtet werden!!!
### morphologische Operationen
Auf Binärbildern kann man auch Min- und Max-Filter anwenden. Diese Operationen heißen hier dann Erosion und Dilatation. Durch Hintereinanderausführen kann man daraus auch die morphologischen Operationen Opening und Closing definieren.

## Workflow
Der klassische Workflow sieht meist vor, dass ein Bild mit Hilfe eines Filters entrauscht wird. Danach werden durch einen weiteren Filter und mathematische Operationen die Kanten des Bildes hervorgehoben. Anschließend wird das Bild mittels Thresholding binärisiert. Schlussendlich wird das Binärbild nochmals bearbeitet.

In [ ]:
import scipy
import skimage
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from tqdm import tqdm
from glob import glob
from tifffile import imread

from scipy import ndimage
from skimage import filters
from skimage.morphology import disk
from skimage.measure import label, regionprops

from stardist import random_label_cmap
from stardist.matching import matching_dataset
from csbdeep.utils import Path
lbl_cmap = random_label_cmap()

In [ ]:
# Damit wir den Effekt von Filtern nachvollziehen können schreiben wir eine kleine Funktion,
# um mehrere Bilder mit verschiedenen Titeln nebeneinander darstellen zu können.
def plot_effect(imgs, labels, cmaps=None):
    cols = len(imgs)
    rows = len(imgs[0])
    if cmaps is None:
        cmaps = [None,] * rows
    fig, axes = plt.subplots(cols, rows, figsize=[16, cols*5])
    for i in range(cols):
        for j in range(rows):
            try:
                ax = axes[i, j]
            except IndexError:
                ax = axes[j]
            if not imgs[i][j] is None:
                ax.imshow(imgs[i][j], cmap=cmaps[j])
                ax.axis("off")
                ax.set_title(labels[i][j])
            else:
                ax.remove()

Wir laden wieder unsere Trainings-Daten und benutzen für den weiteren Verlauf einen Ausschnitt des ersten Bilds. Beim Laden der Bilder transformieren wir die uint16 Bilddaten in den int64 Typ. Was bedeutet das?

In [ ]:
X_glob = sorted(glob('/extdata/readonly/f-prak-v15/e-coli-swarming/train/input/*.tif'))
Y_glob = sorted(glob('/extdata/readonly/f-prak-v15/e-coli-swarming/train/labels/*.tif'))
X = [x.astype(int) for x in list(map(imread, X_glob))]
Y = [x.astype(int) for x in list(map(imread, Y_glob))]
sly = slice(380, 508)
slx = slice(380, 508)
img, lbl = X[0][sly, slx], Y[0][sly, slx]

## Auswahl an Filtern
Im folgenden werden einige Filter auf das Bild angewandt und angezeigt.

In [ ]:
median = filters.median(img, disk(4))
mean = ndimage.filters.uniform_filter(img, 4)
gauss = filters.gaussian(img, 1)
diff_gauss = filters.difference_of_gaussians(img, 1, 6)
black_tophat = ndimage.morphology.black_tophat(img, 4)
white_tophat = ndimage.morphology.white_tophat(img, 4)
plot_effect(
    [
        [img, median, img-median],
        [img, mean, img-mean],
        [img, gauss, img-gauss],
        [img, diff_gauss, img-diff_gauss],
        [img, black_tophat, img-black_tophat],
        [img, white_tophat, img-white_tophat],
    ],
    [
        ["input", "median filter", "input - median filter"],
        ["input", "mean filter", "input - mean filter"],
        ["input", "gauss filter", "input - gauss filter"],
        ["input", "diff gauss filter", "input - diff gauss filter"],
        ["input", "black_tophat filter", "input - black_tophat filter"],
        ["input", "white_tophat filter", "input - white_tophat filter"],
    ]
)

## Histogramm des Bildes
Es ist manchmal auch ganz sinnvoll das Histogramm eines Bildes anzuschauen. Könnt ihr die verschiedenen Bereiche des Histogramms Regionen im Bild zuordnen?

In [ ]:
plt.hist(img.flatten(), bins=50)
None;

## Entrauschen
Wir wählen nun einfach den Gauß-Filter, um das Bild zu entrauschen. Der Gauß-Filter kann nun noch über die Größe Sigma beeinflusst werden. Im Folgenden betrachten wir die entrauschten Bilder für verschiedene Werte für Sigma.

In [ ]:
imgs = []
labels = []
sigmas = list(range(1,10))
for sigma in sigmas:
    gauss = filters.gaussian(img, sigma)
    imgs.append([
        img, gauss,
    ])
    labels.append([
        "input",
        "Gauß Filter, sigma = {}".format(sigma),
    ])
plot_effect(imgs, labels)

### Wahl von Sigma
Aus den obigen Bildern kann man nun einen Wert für Sigma ermittlen. Bei Sigma=1 erscheinen im gefilterten Bild noch recht kleine Strukturen des Hintergrund. Bis Sigma=3 ist das gesamte Bild schon recht verschwommen. Daher wählen wir Sigma=2. Ihr könnt auch andere Filter und andere Sigmas ausprobieren bis ihr ein Ergebnis findet, das euch gefällt.

## Zum Spielen
Was macht der folgende Code?

In [ ]:
# Wir speichern das entrauschte Bild in einer neuen Variable ab, die wir im folgenden weiter verwenden.
img_denoise = filters.gaussian(img.astype(float), 2)
imgs = []
labels = []
sizes = list(range(6, 14))
for size in sizes:
    white_tophat = scipy.ndimage.morphology.white_tophat(img_denoise, size)
    median = skimage.filters.median(white_tophat, disk(5))
    imgs.append([
        img_denoise, white_tophat, median
    ])
    labels.append([
        "denoised input",
        "white tophat with size = {}".format(size),
        "median of tophat"
    ])
plot_effect(imgs, labels)

## Auswahl einer Tophat Filter Size
Wir hatten oben bereits gesehen, dass der white tophat Filter das Innere der Zellen ganz gut hervorgehoben hat, während der Hintergrund des Bildes sehr dunkel wurde. Trotzdem werden beim Tophat Filter auch Strukturen im Hintergrund des Bildes sichtbar. Findet ihr einen Filter mit dem ihr diese Hintergrundstruktur entfernen könnt?

## Vergleich der Histogramme
Im Folgenden wurde ein white tophat mit Size 10 verwendet. Zudem wird ein Bild nach dem Tophat Filter noch mit einem median Filter versehen. Dann werden die Histogramme der Bilder miteinander verglichen.

In [ ]:
tophat = scipy.ndimage.morphology.white_tophat(img_denoise, 10)
tophat_median = skimage.filters.median(tophat, disk(5))
binsize = 50
plt.figure(figsize=[16, 16])
plt.subplot(311)
plt.hist(img_denoise.flatten(), bins=binsize)
plt.subplot(312)
plt.hist(tophat.flatten(), bins=binsize)
plt.subplot(313)
plt.hist(tophat_median.flatten(), bins=binsize)
None;

## Kompletter Ablauf
Die nächste Zelle zeigt die einzelnen Tranformationsschritte bis hin zur Segmentierung mittels Otus Thresholding für verschiedene Abläufe.

In [ ]:
gauss = skimage.filters.gaussian(img, 2)
white_tophat = scipy.ndimage.morphology.white_tophat(gauss, 10)
tophat_median = skimage.filters.median(tophat, disk(5))

thresh_raw = skimage.filters.threshold_otsu(img)
thresh_denoise = skimage.filters.threshold_otsu(gauss)
thresh_tophat = skimage.filters.threshold_otsu(white_tophat)
thresh_median = skimage.filters.threshold_otsu(tophat_median)

binary_raw = label(img > thresh_raw)
binary_denoise = label(gauss > thresh_denoise)
binary_tophat = label(white_tophat > thresh_tophat)
binary_median = label(tophat_median > thresh_median)
plot_effect(
    [
        [img, gauss, white_tophat, tophat_median, binary_median],
        [img, gauss, white_tophat, None, binary_tophat],
        [img, gauss, None, None, binary_denoise],
        [img, None, None, None, binary_raw],
    ],
    [
        ["input", "gauss sigma=2", "white_tophat size=10", "median size=5", "otsu threshold"],
        ["input", "gauss sigma=2", "white_tophat size=10", "median size=5", "otsu threshold"],
        ["input", "gauss sigma=2", "white_tophat size=10", "median size=5", "otsu threshold"],
        ["input", "gauss sigma=2", "white_tophat size=10", "median size=5", "otsu threshold"],
    ],
    [None, None, None, None, lbl_cmap]
)

## Vergleich mit Ground Truth Labeln
Nun betrachten wir einen größeren Ausschnitt des Bildes und vergleichen es mit unseren von Hand annotierten Labels. Wir schreiben eine Funktion, um aus einem Bild nur noch die fertige Segmentierung zu erhalten. Um einen schnellen Eindruck für die Qualität eines Segmentier-Algorithmus zu bekommen, kann man auch vergleichen, wie viele einzelne Objekte entdeckt worden sind. Dazu benutzen wir die Funktion [regionprops](https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.regionprops).

### Aufgabe 2-0
Schreibt die Funktion segment so um, dass sie die Segmentierung verwendet, die ihr für sinnvoll und gut haltet.

In [ ]:
def segment(x):
    gauss = skimage.filters.gaussian(x, 2)
    white_tophat = scipy.ndimage.morphology.white_tophat(gauss, 10)
    thresh = skimage.filters.threshold_otsu(white_tophat)
    binary = white_tophat > thresh
    labelmap = label(binary)
    return labelmap

slx = slice(200, 800)
sly = slice(200, 800)
i = 0
x = X[i][sly, slx]
gt = Y[i][sly, slx]
seg = segment(x)
region_gt = skimage.measure.regionprops(gt)
region_seg = skimage.measure.regionprops(seg)

for name, arr in zip(["Segmentierung", "Labels"], [region_seg, gt]):
    print("Anzahl Objekte in {}: {}".format(name, len(arr)))

plot_effect([[x, seg, gt]], [["input", "seg", "label"]], [None, lbl_cmap, lbl_cmap])

## Aufgabe 2-1
Benutzt die Funktion aus dem ersten Notebook, um die Segmentierung über das Inputbild zu legen. In welchen Regionen funktioniert die Segmentierung gut? Wo ist sie schlecht? Welche Probleme fallen bei der Segmentierung auf?

## Aufagbe 2-2
Schreibt Funktionen um einige Beurteilungskriterien für binäre Klassifizierer zu erhalten. Berechnet diese Kriterien für das Bild und für die Bilder des Test Sets. Im Folgenden ist eine Funktion beschrieben, die die *true positives* einer Segmentierung und zugehörigem Label berechnet. Diese Funktion arbeitet pixelweise. Eure Funktionen sollen auch nur pixelweise vergleiche anstellen.

In [ ]:
def tp(seg, lbl):
    """
    Berechnet die true positives zwischen seg und lbl. Dabei ist lbl die Ground Truth.
    seg und lbl sollen Binärbilder vom Typ bool sein.
    """
    # Da die Bilder Binärbilder sind, können wir boolesche Operationen ausführen.
    # Ein Pixel ist true positive, wenn es in seg 1 und in lbl 1 ist,
    # daher können wir die logische Operation und verwenden und alle übriggebliebenen
    # 1 zählen. Das sind dann die true positives
    truepos = np.logical_and(seg, lbl)
    return np.sum(truepos)

# Wir transformieren zunächst unsere lablemaps in bool Arrays
seg_b = seg > 0
gt_b = gt > 0
print(tp(seg_b, gt_b))